In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import requests
from bs4 import BeautifulSoup

%matplotlib inline

In [58]:
url = 'http://www.boxofficemojo.com/movies/?page=intl&id=marvel2016.htm'
#url = 'http://www.boxofficemojo.com/movies/?page=intl&id=disney2016.htm'
#url = 'http://www.boxofficemojo.com/movies/?page=intl&id=junglebook2015.htm'
#url = 'http://www.boxofficemojo.com/movies/?page=intl&id=deadpool2016.htm'


resp = requests.get(url)
page = resp.text
soup = BeautifulSoup(page)

C:\Python34\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "html.parser")

  markup_type=markup_type))


In [59]:
toptopTable = topTable[1].find_all('td')
type(toptopTable)

bs4.element.ResultSet

In [60]:
box = soup.find_all('table')

for i in box:
    print(len(i))
    
len(box)

6
2
2
2
1
4
1
94


8

In [61]:
topBox = box[5]
topBox

<table bgcolor="#dcdcdc" border="0" cellpadding="4" cellspacing="1" width="95%"><tr bgcolor="#ffffff"><td align="center" colspan="2"><font size="4"><font size="3">Domestic Total as of Jul. 17, 2016:</font> <b>$406,573,683</b></font></td></tr><tr bgcolor="#ffffff"><td valign="top">Distributor: <b><a href="/studio/chart/?studio=buenavista.htm">Buena Vista</a></b></td><td valign="top">Release Date: <b><nobr><a href="/schedule/?view=bydate&amp;release=theatrical&amp;date=2016-05-06&amp;p=.htm">May 6, 2016</a></nobr></b></td></tr><tr bgcolor="#ffffff"><td valign="top">Genre: <b>Action / Adventure</b></td><td valign="top">Runtime: <b>2 hrs. 27 min.</b></td></tr><tr bgcolor="#ffffff"><td valign="top">MPAA Rating: <b>PG-13</b></td><td valign="top">Production Budget: <b>$250 million</b></td></tr></table>

## Here is section where we get top box information

In [62]:
for element in topBox.find_all('td'):
    print(element.text)
    print('----------')

Domestic Total as of Jul. 17, 2016: $406,573,683
----------
Distributor: Buena Vista
----------
Release Date: May 6, 2016
----------
Genre: Action / Adventure
----------
Runtime: 2 hrs. 27 min.
----------
MPAA Rating: PG-13
----------
Production Budget: $250 million
----------


In [63]:
def findRelevantData(tableElement):
    listOfTableData = tableElement.find_all('td')
    distributorText = listOfTableData[2].text.split(':')[1].strip()
    
    # convert to actual number
    prodBudget = listOfTableData[-1].text.split(':')[1].strip()
    mpaa_rating = listOfTableData[-2].text.split(':')[1].strip()
    
    # convert this to datetime
    releaseDate = listOfTableData[3].text.split(':')[1].strip()
    
    runTime = listOfTableData[-3].text.split(':')[1].strip()
    genre = listOfTableData[4].text.split(':')[1].strip()
    
    return distributorText, releaseDate, genre, runTime, mpaa_rating, prodBudget

print(findRelevantData(topBox))

('May 6, 2016', 'Action / Adventure', '2 hrs. 27 min.', '2 hrs. 27 min.', 'PG-13', '$250 million')


## Country and revenue data

In [64]:
countryData = box[-1]
countryData

<table border="0" cellpadding="3" cellspacing="1"><tr bgcolor="#dcdcdc"><td align="center"><font size="2"><a href="/movies/?page=intl&amp;id=marvel2016.htm&amp;sort=country&amp;order=ASC&amp;p=.htm">Country<br>(click to view weekend breakdown)</br></a></font></td><td align="center"><font size="2"><a href="/movies/?page=intl&amp;id=marvel2016.htm&amp;sort=studio&amp;order=ASC&amp;p=.htm">Dist.</a></font></td><td align="center"><font size="2"><a href="/movies/?page=intl&amp;id=marvel2016.htm&amp;sort=ReleaseDate&amp;order=DESC&amp;p=.htm">Release<br>Date</br></a></font></td><td align="center"><font size="2"><a href="/movies/?page=intl&amp;id=marvel2016.htm&amp;sort=opengross&amp;order=DESC&amp;p=.htm">Opening<br>Wknd</br></a></font></td><td align="center"><font size="2"><a href="/movies/?page=intl&amp;id=marvel2016.htm&amp;sort=percoftotal&amp;order=DESC&amp;p=.htm">% of<br>Total</br></a></font></td><td align="center" colspan="2"><font size="2"><a href="/movies/?page=intl&amp;id=marvel20

In [65]:
countryDataList = []
for row in countryData.find_all('tr'):
    
    rowData = []

    rowData = [x.text for x in row.find_all('td')]
    countryDataList.append(rowData)
    
header = countryDataList[0]
data = countryDataList[3:]

# this section will take the header of country information and then split
# last element and update it and add it back to header
total_gross = header[-1].split('/')[0].strip().replace(' ', '_')
total_gross_as_of = total_gross + '_date'
total_gross_as_of

header.pop(-1)
header.extend([total_gross, total_gross_as_of])

In [66]:
import pandas as pd

df = pd.DataFrame(data)
df.columns = header

In [67]:
topBoxHeader = ['UsDist','UsRelease','Genre','Runtime','MpaaRating','ProdBudget']
for element in zip(topBoxHeader, findRelevantData(topBox)):
    df[element[0]] = element[1]

In [69]:
df

,Country(click to view weekend breakdown),Dist.,ReleaseDate,OpeningWknd,% ofTotal,Total_Gross,Total_Gross_date,UsDist,UsRelease,Genre,Runtime,MpaaRating,ProdBudget
0,Argentina,Disney,5/5/16,"$3,555,979",39.5%,"$9,005,829",7/3/16,"May 6, 2016",Action / Adventure,2 hrs. 27 min.,2 hrs. 27 min.,PG-13,$250 million
1,Australia,Disney,4/28/16,"$10,622,362",42.2%,"$25,199,942",7/10/16,"May 6, 2016",Action / Adventure,2 hrs. 27 min.,2 hrs. 27 min.,PG-13,$250 million
2,Austria,Disney,1/1/01,-,-,"$2,841,356",7/10/16,"May 6, 2016",Action / Adventure,2 hrs. 27 min.,2 hrs. 27 min.,PG-13,$250 million
3,Belgium,Disney,4/27/16,-,-,"$2,484,111",7/10/16,"May 6, 2016",Action / Adventure,2 hrs. 27 min.,2 hrs. 27 min.,PG-13,$250 million
4,Brazil,Disney,4/28/16,"$11,984,952",29.6%,"$40,521,418",7/3/16,"May 6, 2016",Action / Adventure,2 hrs. 27 min.,2 hrs. 27 min.,PG-13,$250 million
5,Bulgaria,Forum Films,4/29/16,"$185,804",36%,"$516,255",6/19/16,"May 6, 2016",Action / Adventure,2 hrs. 27 min.,2 hrs. 27 min.,PG-13,$250 million
6,China,Disney,5/6/16,"$93,640,448",49.2%,"$190,429,000",6/12/16,"May 6, 2016",Action / Adventure,2 hrs. 27 min.,2 hrs. 27 min.,PG-13,$250 million
7,Czech Republic,Falcon,5/5/16,"$584,798",40.1%,"$1,459,184",6/19/16,"May 6, 2016",Action / Adventure,2 hrs. 27 min.,2 hrs. 27 min.,PG-13,$250 million
8,Denmark,Disney,4/28/16,-,-,"$3,078,912",7/10/16,"May 6, 2016",Action / Adventure,2 hrs. 27 min.,2 hrs. 27 min.,PG-13,$250 million
9,Finland,Disney,4/27/16,"$235,310",16.5%,"$1,427,013",7/10/16,"May 6, 2016",Action / Adventure,2 hrs. 27 min.,2 hrs. 27 min.,PG-13,$250 million


In [27]:
df.to_csv('jungleBook.csv', encoding='utf-8')